Copyright 2024 shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Gemini Pro Vision Image 분석

Feedback : shins777@gmail.com. 

이 Colab 은 다양한 비즈니스 요구사항에 적용할수 있는 이미지 분석에 대한 예제입니다.
Gemini Pro vision을 활용해서 이미지를 분석합니다.

* 코드는 Gemini Pro API 기반으로 처리하며 그에 따른 API는 아래 링크 참고하세요.  
    * 자세한 정보는 [README.md](https://github.com/shins777/google_gen_ai_sample/blob/main/notebook/gemini/README.md) 파일 참고하세요.

##### 참고: 이곳에서 사용되는 이미지는 교육목적으로 무료 웹 상에서 다운로드 받은 이미지입니다. 상업적인 용도로 사용하지 말아주세요. 혹시 저작권의 문제가 있다면 위의 이메일로 연락 부탁드립니다.



In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


### GCP 사용자 인증 / 환경설정

GCP 인증방법은 아래와 URL 정보를 참고하여 GCP에 접근 하는 환경을 구성해야 합니다. 
* https://cloud.google.com/docs/authentication?hl=ko
* 자세한 정보는 [README.md](https://github.com/shins777/google_gen_ai_sample/blob/main/notebook/gemini/README.md) 파일 참고하세요.

In [2]:
#  아래 코드는 Colab 환경에서만 실행해주세요. 다른 환경에서는 동작하지 않습니다.
# from google.colab import auth
# auth.authenticate_user()

### GCP 프로젝트 및 리전 설정
본인의 GCP 환경에 맞게 아래 설정을 구성하세요.  
* 구글의 최신버전인 gemini pro 사용을 권고드립니다.   
* 만일, 기본 버전 text bison 을 사용하려한다면, 참조하는 class 가 다르므로 주의하세요.  
* 현재 Gemini는 한국리전(asia-northeast3)을 통해서 접근이 가능합니다.

In [3]:
model_name="gemini-pro-vision"
project="ai-hangsik"
location="asia-northeast3"

#### Google Generative AI를 사용하기 위한 aiplatform initialize.

In [4]:
from google.cloud import aiplatform
aiplatform.init(project=project, location = location)

### Gemini Pro vision - Multimodal

이미지 base64 인코딩을 위한 함수

In [5]:
import base64
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

def get_obj_content(coding='utf-8', obj_loc=""):
    with open(obj_loc, 'rb') as f:
        obj_data = base64.b64encode(f.read()).decode(coding)
        return obj_data

### Image 분석
* Image 분석시 아래 가이드를 참고하세요. 
    * Image MIME type : PNG, JPEG.
    * 상세 요구서항 : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts?hl=ko#image-requirements  
    
* 예제에서 제시된 이미지는 Gemini Pro Vision 테스트용 이외의 목적으로 상업적으로 사용하지 말아주세요.

#### 주식 챠트 이미지 분석

In [6]:
def analyze_image(file_path:str, prompt:str)->str:
  """
  Image 분석하는 예제
  file_path : 분석 대상 이미지 path.
  prompt : 대상이미지에서 분석하고자 하는 프롬프트.
  """
  img_data = get_obj_content(obj_loc=file_path)
  image_obj = Part.from_data(data=base64.b64decode(img_data), mime_type="image/png")

  model = GenerativeModel(model_name)
  responses = model.generate_content(
                                    [image_obj, prompt],
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.4,
        "top_p": 1,
        "top_k": 32
    },
    safety_settings=[],
    stream=False,
  )
  return responses.text

In [7]:
file_path = "../../../contents/images/Kospi.png"

prompt = """이미지에 대해서 아래 내용을 중심으로 자세하게 설명해주세요.
1. 몇일날짜의 정보입니다.
2. 52주 최고가는 얼마인가요 ?
3. 그래프가 생성된 다음날의 상황은 어떻게 예측하나요 ? 이유도 알려주세요.
4. 이 그래프는 일반기업의 그래프인가요 ?
5. 이 그래프는 어떤 데이터를 기반으로 만들어졌나요 ?
"""

result = analyze_image(file_path, prompt)
print(result)

 1. 2023년 3월 7일 화요일 오후 6시 5분 기준 정보입니다.
2. 52주 최고가는 2,694.80입니다.
3. 다음날 상황은 상승할 것으로 예상합니다. 이유는 오늘의 상승장세와 52주 최고가에 근접했기 때문입니다.
4. 이 그래프는 일반기업의 그래프가 아닙니다. KOSPI 지수는 한국거래소에 상장된 모든 주식의 가격을 반영하는 지수입니다.
5. 이 그래프는 한국거래소에서 제공하는 데이터를 기반으로 만들어졌습니다.


#### 이미지 비교 분석
아래 예제는 여러개의 이미지를 비교해서 분석해주는 예제입니다.

In [8]:

def analyze_image(file_path1:str, file_path2:str, prompt:str)->str:

  img_data1 = get_obj_content(obj_loc=file_path1)
  image_obj1 = Part.from_data(data=base64.b64decode(img_data1), mime_type="image/png")

  img_data2 = get_obj_content(obj_loc=file_path2)
  image_obj2 = Part.from_data(data=base64.b64decode(img_data2), mime_type="image/png")

  model = GenerativeModel(model_name)

  responses = model.generate_content(
                                    [image_obj1, image_obj2, prompt],
                                    generation_config={
                                        "max_output_tokens": 2048,
                                        "temperature": 0.4,
                                        "top_p": 1,
                                        "top_k": 32
                                    },
                                    safety_settings=[],
                                    stream=False,
  )


  return responses.text

In [9]:
file_path1 = "../../../contents/images/liver1.png"
file_path2 = "../../../contents/images/liver2.png"

prompt = """ 두개의 이미지를 각각 분석해주고, 두개의 이미지가 다른 점이 무엇인지 상세하게 설명해주세요.
"""

result = analyze_image(file_path1,file_path2, prompt)
print(result)

 두 개의 이미지는 모두 복부 초음파 검사 이미지입니다. 첫 번째 이미지는 정상적인 간을 보여주고 두 번째 이미지는 담석이 있는 담낭을 보여줍니다.

첫 번째 이미지에서 간은 균질한 에코원성을 보이고 있으며, 담낭은 얇은 벽과 무에코성의 내용물을 보입니다. 두 번째 이미지에서 담낭은 두껍고 불규칙한 벽을 보이며, 담낭 내에 다수의 음영이 보입니다. 이러한 음영은 담석입니다. 담석은 담낭에 칼슘, 콜레스테롤 및 기타 물질이 축적되어 형성됩니다. 담석은 담낭에 통증, 발열, 오한 및 메스꺼움을 유발할 수 있습니다. 담석이 담관을 차단하면 황달을 유발할 수도 있습니다.

두 개의 이미지를 비교해보면 담석이 있는 담낭은 정상적인 담낭보다 두껍고 불규칙한 벽을 보이며, 담낭 내에 음영이 보인다는 것을 알 수 있습니다. 이러한 차이는 담석이 담낭에 미치는 영향을 보여줍니다.


#### 수학 문제 해석
* 아래는 수학문제 풀이에 관한 신문 기사입니다. 테스트를 위해서 가져온 데이터 입니다. 다른 용도로 사용하지 말아주세요.  
* https://www.donga.com/news/Society/article/all/20090922/8812199/1

In [10]:
def analyze_image(obj):
  
  model = GenerativeModel(model_name)
  
  responses = model.generate_content(
    [obj, """이미지 안에 있는 문제를 먼저 그대로 작성해주세요.
    해당 문제에 대해서 답을 구하는 절차를 최대한 상세하게 설명해주세요.
    계산결과 늘어난 길이는 얼마나 되나요 ?

     """],

    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.4,
        "top_p": 1,
        "top_k": 32
    },
    safety_settings=[],
    stream=True,
    )

  for response in responses:
      print(response.text, end="")

In [11]:
file_path = "../../../contents/images/math.png"

img_data = get_obj_content(obj_loc=file_path)
image_obj = Part.from_data(data=base64.b64decode(img_data), mime_type="image/png")

analyze_image(image_obj)

 **[문제]**
그림과 같이 가로, 세로의 길이가 각각 5m, 4m인 직사각형 모양의 목장을 가로, 세로의 길이를 똑같은 길이만큼 늘렸더니 처음보다 넓이가 10m²만큼 늘었다. 이때 늘어난 길이를 구하여라.

**[해결]**
1. 가로, 세로의 길이를 각각 x만큼 늘렸다고 하면, 넓이는 (5+x)(4+x)가 된다.

2. 처음 넓이에 10을 더한 값은 (5+x)(4+x) = 20 + 9x + 4x + x²이다.

3. 20 + 9x + 4x + x² = 30이므로, x² + 13x - 10 = 0이다.

4. (x+1)(x-10) = 0이므로, x = -1 또는 x = 10이다.

5. x가 음수가 될 수 없으므로, x = 10이다.

**[답]**
늘어난 길이는 10m이다.